Opis:
    
klasa na metody których używam w Senderze i Listenerze

In [7]:
import numpy as np
import binascii
from bitarray import bitarray

class Encoder(object):
    DICT_4B5B = {'0000': '11110',
                 '0001': '01001',
                 '0010': '10100',
                 '0011': '10101',
                 '0100': '01010',
                 '0101': '01011',
                 '0110': '01110',
                 '0111': '01111',
                 '1000': '10010',
                 '1001': '10011',
                 '1010': '10110',
                 '1011': '10111',
                 '1100': '11010',
                 '1101': '11011',
                 '1110': '11100',
                 '1111': '11101'}
    @staticmethod
    def to_4b5b(b):
        """
            przyjmuje binarnego stringa i zwraca zakodowanego 4b5b binarnego stringa
        """
        if len(b) % 4 != 0:
            raise Exception("nieprawidłowy string do zakodowania 4b5b")
        
        return ''.join([Encoder.DICT_4B5B[b[i:(i+4)]] for i in xrange(0, len(b), 4)])
    
    @staticmethod
    def from_4b5b(l):
        """
            przyjmuje binarnego stringa i zwraca zdekodowanego 4b5b binarnego stringa
        """
        if len(l) % 5 != 0:
            raise Exception("nieprawidłowy kod 4b5b")
        
        res = []
        for i in xrange(0, len(l), 5):
            a = l[i:(i+5)]
            found = False
            for k,v in Encoder.DICT_4B5B.items():
                if v == a:
                    res.append(k)
                    found = True
                    break
            if not found:
                raise Exception("zły fragment 4b5b")
        return ''.join(res)
            
    @staticmethod
    def mac_to_bits(m):
        """
            zwraca binarnego stringa równoważnego podanemu adresowi MAC
        """
        m = m.replace(':', '')
        m = m.upper()
        res = []
        for c in m:
            if ord(c) >= ord('0') and ord(c) <= ord('9'):
                res.append(np.binary_repr(ord(c) - ord('0'), 4))
            else:
                res.append(np.binary_repr(ord(c) - ord('A') + 10, 4))
        res = ''.join(res)
        return res
        
    @staticmethod
    def crc32(s):
        """
            Zwraca unsigned inta reprezentującego crc32
        """
        x = binascii.crc32(s)
        if x < 0: 
            x = (1 << 31) - x
        return x
    
    @staticmethod
    def string_from_bits(s):
        b = bitarray(s)
        return b.tostring()
    
    @staticmethod
    def string_to_bits(s):
        b = bitarray()
        b.fromstring(s)
        return b.to01()
    
    @staticmethod
    def restore_message(s):
        """
            bierze najsurowszą wiadomość, czyli z jakimś fragmentem preambuły, 
            jakimś sufiksem z -1 i jakimś fragmentem zbędnego sufiksu
            
            zwraca wiadomość oczyszczoną z wyżej wymienionych rzeczy
        """
        i = 0
        while i < len(s) and s[i] != -1:
            i += 1
        s = s[0:i]
        
        i = 0
        for i in xrange(len(s)): 
            if s[i:(i+5)] == [0, 1, 1, 1, 1]:
                s = s[(i+5):]
                break
        for i in xrange(len(s) - 1, 0, -1):
            if s[i] == 1:
                s = s[:i]
                return s
        raise Exception("nieczytelna wiadomość")
    
    @staticmethod
    def get_frames(target_mac, source_mac, data, l=1499):
        """
            generuje listę ramek o danej długości
        """
        suf = len(data) % l
        data = data + ("0" * (l - (len(data) % l)))
        return [Encoder.create_frame(target_mac, 
                                     source_mac, 
                                     data[i:(i+l)] + ("1" if (i + l == len(data)) else "0"),
                                     suf if (i + l == len(data)) else l)
                for i in xrange(0, len(data), l)]
    
    @staticmethod
    def create_frame(target_mac, source_mac, data, real_len):
        """
            generuje jedną ramkę
        """
        prefix = "1010" * 63 + "1111"
        suffix = "1" + "0" * 255
        data2 = (Encoder.mac_to_bits(target_mac) + 
                 Encoder.mac_to_bits(source_mac) + 
                 np.binary_repr(real_len, 16) + 
                 data)
        data2 = data2 + np.binary_repr(Encoder.crc32(data2), 32)
        new_data = Encoder.to_4b5b(data2)
        return prefix + new_data + suffix
    
    @staticmethod
    def get_data_from_message(ss):
        """
            bierze listę bitów - wiadomość bez zbędnego prefiksu i sufiksu i ją odkodowuje
        """
        ss = [chr(ord("0") + x) for x in ss]
        s = ''.join(ss)
        s = Encoder.from_4b5b(s)
        checksum = int(s[(-32):], 2)
        s = s[0:-32]
        real_checksum = Encoder.crc32(s)
        
        if real_checksum != checksum:
            raise Exception("niezgodna czeksuma")
        end_bit = s[-1]
        t_mac = s[0:48]
        m_mac = s[48:96]
        s = s[96:-1]
        l = int(s[:16], 2)
        s = s[16:]
        return (s[:l], end_bit)